### building a question and answer chatbot with graph db

In [10]:
NEO4J_URI="neo4j+s://238e8e84.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="iMl5YUGpHqyBYEqqx-n1-KwGEn9-COCgJrQIXCxKyoU"

In [11]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [12]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

In [13]:
movie_review="""
LOAD CSV WITH HEADERS FROM
"https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv" AS row

MERGE (m:Movie {id: toInteger(row.movieId)})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)

FOREACH (director IN split(row.director, "|") |
    MERGE (p:Person {name: trim(director)})
    MERGE (p)-[:DIRECTED]->(m)
)

FOREACH (actor IN split(row.actor, "|") |
    MERGE (p:Person {name: trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m)
)

FOREACH (genre IN split(row.genres, "|") |
    MERGE (g:Genre {name: trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g)
)

"""

graph.query(movie_review)


[]

In [14]:
movie_review

'\nLOAD CSV WITH HEADERS FROM\n"https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv" AS row\n\nMERGE (m:Movie {id: toInteger(row.movieId)})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\n\nFOREACH (director IN split(row.director, "|") |\n    MERGE (p:Person {name: trim(director)})\n    MERGE (p)-[:DIRECTED]->(m)\n)\n\nFOREACH (actor IN split(row.actor, "|") |\n    MERGE (p:Person {name: trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m)\n)\n\nFOREACH (genre IN split(row.genres, "|") |\n    MERGE (g:Genre {name: trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g)\n)\n\n'

In [15]:
graph.query(movie_review)

[]

In [16]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {name: STRING, YOB: STRING, POB: STRING}
company {name: STRING}
farmer {name: STRING, YOB: INTEGER, POB: STRING}
country {name: STRING}
datascince {name: STRING, YOB: INTEGER, POB: STRING}
person {name: STRING, born: INTEGER}
movies {name: STRING, released: INTEGER}
User {name: STRING}
Movie {released: DATE, id: INTEGER, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)


In [17]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [18]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000203CD3D6530>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000203CD3D6440>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [19]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x00000203CCDB1F60>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000203CD3D6530>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000203CD3D6440>, model_name=

In [20]:
response=chain.invoke({"query":"who is the director of Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:DIRECTED]->(m:Movie) WHERE m.title = "Casino" RETURN p.name
Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'who is the director of Casino',
 'result': 'Martin Scorsese is the director of Casino.'}

In [23]:
response=chain.invoke({"query":"total number of artist"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Person) RETURN COUNT(n)
Full Context:
[{'COUNT(n)': 290}]

> Finished chain.


{'query': 'total number of artist', 'result': "I don't know the answer."}